This notebook constructs the user conversion funnel and defines churn-related indicators. It tracks users across key stages of the product lifecycle (activated, engaged, intent, converted) and creates churn flags based on inactivity and dropout rules. 

## Load and Clean Dataset

In [1]:
import pandas as pd

df = pd.read_csv("/Users/quynhnguyen/Documents/User Churn Project/Data/2019-Oct.csv")
df['event_time'] = pd.to_datetime(df['event_time'])
df = df.sort_values(['user_id', 'event_time'])

df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
31964969,2019-10-23 20:04:08+00:00,view,7002639,2053013560346280633,kids.carriage,bumbleride,769.65,33869381,d83dc524-8a2c-4780-bbe0-f8aee03b54df
15808538,2019-10-13 00:13:46+00:00,view,10600284,2053013561554240247,NaN,NaN,0.00,64078358,6183edb9-f592-4bb8-9913-67bb4711694d
2420043,2019-10-02 21:43:00+00:00,view,22200103,2053013558785999453,NaN,NaN,15.77,183503497,884233e8-8b9f-4970-808b-4e1c81f8a5fc
4799279,2019-10-04 17:44:37+00:00,view,6902133,2053013560312726199,furniture.living_room.chair,joie,143.89,184265397,2c5d0468-32e1-42f7-9b4e-b17c1d387247
4800257,2019-10-04 17:45:18+00:00,view,6902133,2053013560312726199,furniture.living_room.chair,joie,143.89,184265397,2c5d0468-32e1-42f7-9b4e-b17c1d387247


In [2]:
# Standardize Timezone
df['event_time'] = pd.to_datetime(df['event_time'], utc=True)
df['event_time'] = df['event_time'].dt.tz_convert(None)

# Retain necessary events for funnel
df = df[df['event_type'].isin(['view', 'cart', 'purchase', 'remove_from_cart'])]

# Drop invalid users
df = df[df['price'] > 0]

# Drop users with only 1 event
user_event_counts = df['user_id'].value_counts()
valid_users = user_event_counts[user_event_counts >= 2].index
df = df[df['user_id'].isin(valid_users)]

# Create a standard session-level timestamp
df['date'] = df['event_time'].dt.date
df['week'] = df['event_time'].dt.to_period('W').astype(str)
df['month'] = df['event_time'].dt.to_period('M').astype(str)

# Fill missing data
df['category_code'] = df['category_code'].fillna('unknown')
df['brand'] = df['brand'].fillna('unknown')

# Save the clean dataset
clean_path = "/Users/quynhnguyen/Documents/User Churn Project/Data/events_clean_oct.csv"
df.to_csv(clean_path, index=False)

In [3]:
df.shape
df['event_type'].value_counts()


event_type
view        40019770
cart          926295
purchase      742809
Name: count, dtype: int64

## Build User-Level Table

In [4]:
# Aggregate per user
user_stats = df.groupby('user_id').agg(
    first_seen=('event_time', 'min'),
    last_seen=('event_time', 'max'),
    n_events=('event_time', 'count'),
    n_views=('event_type', lambda x: (x == 'view').sum()),
    n_carts=('event_type', lambda x: (x == 'cart').sum()),
    n_purchases=('event_type', lambda x: (x == 'purchase').sum())
).reset_index()

user_stats.head()


,user_id,first_seen,last_seen,n_events,n_views,n_carts,n_purchases
0,184265397,2019-10-04 17:44:37,2019-10-15 17:19:28,6,6,0,0
1,200673532,2019-10-10 15:02:36,2019-10-13 14:26:21,4,4,0,0
2,205053188,2019-10-09 10:30:19,2019-10-09 10:30:44,2,2,0,0
3,208669541,2019-10-04 05:49:14,2019-10-04 05:49:46,2,2,0,0
4,209714031,2019-10-20 18:29:45,2019-10-30 05:45:56,22,22,0,0


## Define Funnel Stages

In [5]:
user_stats['activated'] = user_stats['n_views'] >= 1
user_stats['engaged'] = user_stats['n_views'] >= 3
user_stats['intent'] = user_stats['n_carts'] >= 1
user_stats['converted'] = user_stats['n_purchases'] >= 1

## 7-Day Retention

In [6]:
df = df.merge(user_stats[['user_id','first_seen']], on='user_id', how='left')
df['days_since_first'] = (df['event_time'] - df['first_seen']).dt.days

retained_7d = df[df['days_since_first'] >= 7].groupby('user_id').size().reset_index(name='retained_7d_flag')
retained_7d['retained_7d'] = 1

user_stats = user_stats.merge(retained_7d[['user_id','retained_7d']], on='user_id', how='left')
user_stats['retained_7d'] = user_stats['retained_7d'].fillna(0).astype(int)


## Churn (30-day inactivity)

In [7]:
analysis_date = df['event_time'].max()
user_stats['days_since_last'] = (analysis_date - user_stats['last_seen']).dt.days
user_stats['churn_30d'] = (user_stats['days_since_last'] >= 30).astype(int)

## Inspect Funnel & Churn

In [8]:
user_stats[['activated','engaged','intent','converted','retained_7d','churn_30d']].mean()

activated      0.999978
engaged        0.830227
intent         0.144626
converted      0.148947
retained_7d    0.363892
churn_30d      0.015542
dtype: float64

## Save for SQL & Dashboard

In [9]:
user_stats.to_csv("/Users/quynhnguyen/Documents/User Churn Project/Data/user_funnel_churn_table_oct.csv", index=False)

In [10]:
user_stats[['activated','engaged','intent','converted','retained_7d','churn_30d']].mean()

activated      0.999978
engaged        0.830227
intent         0.144626
converted      0.148947
retained_7d    0.363892
churn_30d      0.015542
dtype: float64